# Scraping chilling effect on press freedom from CNN

## 2.1 Contents<a id='2.1_Contents'></a>
* [2.1 Importing Relevant Tools](#2.1_Importing)
* [2.2 Defining the Request](#2.2_URL)
* [2.3 Grab | Date](#2.4_scrape_date)
* [2.4 Grab | Header](#2.4_scrape_header)
* [2.5 Grab | Content](#2.4_scrape_content)
* [2.6 Clean | Send to DataFrame](#2.5_review)
* [2.7 Save](#2.6_save)


**Importing Relevant Tools**<a id='2.1_Importing'></a>

In [157]:
import json
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
import csv
import re
import pickle

%reload_ext watermark

In [158]:
#the below needs to be reviewed for all websites; notably the time format

from datetime import date
from datetime import datetime
today = date.today()
d = today.strftime("%m-%d-%y")

For reference.

In [159]:
%watermark -d -t -v -p pandas

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

pandas: 1.1.3



**Defining the Request**<a id='2.2_URL'></a>

In [160]:
url = 'https://www.cnn.com/2020/07/03/media/hong-kong-media-freedom-national-security/index.html'.format(d)
url

'https://www.cnn.com/2020/07/03/media/hong-kong-media-freedom-national-security/index.html'

In [161]:
html = requests.get(url)

In [162]:
bsobj = soup(html.content,'lxml')
# bsobj

**Grab | Date**<a id='2.4_scrape_date'></a>

In [163]:
for date in bsobj.find('div',{'class':'metadata__info js-byline-images'}):
    print(date.text.strip())

By Hadas Gold, CNN Business

Updated 9:53 AM ET, Fri July 3, 2020


A lot to clean on this; to be done below.

**Grab | Header**<a id='2.4_scrape_header'></a>

In [164]:
for title in bsobj.findAll("h1"):
    print(format(title.text))

Hong Kong's security law could have a chilling effect on press freedom


Confirmed that's the title.

**Grab | Content**<a id='2.4_scrape_content'></a>

In [165]:
for news in bsobj.findAll('div',{'class':'l-container'}):
    print(news.text.strip())

Hong Kong's security law could have a chilling effect on press freedomBy Hadas Gold, CNN BusinessUpdated 9:53 AM ET, Fri July 3, 2020 JUST WATCHEDHong Kong official: National security law will be 'clean break'ReplayMore Videos ...MUST WATCH (16 Videos)Hong Kong official: National security law will be 'clean break'Inside the logistics system behind AlibabaSee rescue effort to save dozens of puppies from 'mystery box' online saleWestern brands slammed in China over Xinjiang stanceHong Kong media tycoon's long battle for freedomFacebook: Chinese hackers targeted Uyghurs living in USAustralia's wine industry battered as relations with China sour US-China tech rivalry will likely continue under Biden presidency. Here's why See Jack Ma's first public appearance in monthsChina's economy grows 2.3% in 2020Trump administration dials up US-China tech tensionsHow China's Ant Group built a $17 trillion payments machineHSBC may have to choose between China and the WestChina censors a lot, from Winn

Looks like `— Jessie Yeung, Ivan Watson and James Griffiths contributed to this article.` will need to be removed below.

**Clean | Send to DataFrame**<a id='2.5_review'></a>

First the date from string to a datetime object.

In [166]:
for day in bsobj.find('div',{'class':'metadata__info js-byline-images'}):
    day_pub = day.text.strip()

In [167]:
day_pub

'Updated 9:53 AM ET, Fri July 3, 2020'

In [168]:
day_pub = re.sub('Updated 9:53 AM ET, Fri ','',day_pub)
day_pub = re.sub(', ',' ',day_pub)
day_pub = re.sub('July','07',day_pub)
day_pub = re.sub(' ','-',day_pub)
day_pub = datetime.strptime(day_pub, '%m-%d-%Y').date()
day_pub

datetime.date(2020, 7, 3)

In [169]:
df_date = pd.DataFrame([day_pub])

In [170]:
type(df_date)

pandas.core.frame.DataFrame

In [171]:
df_date

,0
0,2020-07-03


Now the title.

In [172]:
for title_s in bsobj.findAll("h1"):
    title_list = format(title_s.text)

In [173]:
df_title = pd.DataFrame([title_list])

In [174]:
df_title

,0
0,Hong Kong's security law could have a chilling...


In [175]:
type(df_title)

pandas.core.frame.DataFrame

These two items are manually added.

In [177]:
country = 'US'
df_country = pd.DataFrame([country])
source = 'CNN'
df_source = pd.DataFrame([source])
file_name = 'cnn_11'
df_file_name = pd.DataFrame([file_name])


Finally, the news.

In [178]:
news1 = []
for news in bsobj.findAll('div',{'class':'l-container'}):
    news1.append(news.text.strip())

In [179]:
news1

['Hong Kong\'s security law could have a chilling effect on press freedomBy Hadas Gold, CNN BusinessUpdated 9:53 AM ET, Fri July 3, 2020 JUST WATCHEDHong Kong official: National security law will be \'clean break\'ReplayMore Videos ...MUST WATCH (16 Videos)Hong Kong official: National security law will be \'clean break\'Inside the logistics system behind AlibabaSee rescue effort to save dozens of puppies from \'mystery box\' online saleWestern brands slammed in China over Xinjiang stanceHong Kong media tycoon\'s long battle for freedomFacebook: Chinese hackers targeted Uyghurs living in USAustralia\'s wine industry battered as relations with China sour US-China tech rivalry will likely continue under Biden presidency. Here\'s why See Jack Ma\'s first public appearance in monthsChina\'s economy grows 2.3% in 2020Trump administration dials up US-China tech tensionsHow China\'s Ant Group built a $17 trillion payments machineHSBC may have to choose between China and the WestChina censors a

There's a lot to delete at the top & bottom. Let's start at the bottom.

In [180]:
len(news1)

11

In [181]:
n = 7
news1 = news1[:len(news1)-n]

In [182]:
news1[-1]

'London (CNN Business)Hong Kong insists its vibrant community of journalists has nothing to fear from the national security legislation China imposed on the city this week. But press freedom advocates worry about creeping self-censorship, and there are signs that the new law may already be having a chilling effect.In what was once seen as a safe haven of free speech for local and foreign news organizations operating in the region, journalists and their sources are growing increasingly wary that as China takes a firmer grip on Hong Kong, they could be prosecuted. The new law applies to any person in Hong Kong, both locals and foreign nationals. It criminalizes actions like calling for Hong Kong independence, or working with a foreign entity "to incite hatred" toward the Chinese government. A new enforcement committee will "strengthen the management" of NGOs and media working in Hong Kong. The police will also have new powers to search premises, wiretap suspects and order people to "dele

The meat is now at the end; let's reassign.

In [183]:
news1 = news1[-1]

In [184]:
news1 = re.sub('— Jessie Yeung, Ivan Watson and James Griffiths contributed to this article.','',news1)

In [185]:
df_news = pd.DataFrame([news1])

In [186]:
type(df_news)

pandas.core.frame.DataFrame

In [187]:
df_news

,0
0,London (CNN Business)Hong Kong insists its vib...


In [188]:
df_news.columns = ['Article']

In [189]:
df_news.head()

,Article
0,London (CNN Business)Hong Kong insists its vib...


**Bringing it together.**<a id='2.5_bit'></a>

In [190]:
df_11_cnn = pd.concat([df_file_name,df_date,df_source,df_country,df_title,df_news],axis = 1, ignore_index=False)

In [191]:
df_11_cnn.columns = ['file_name','date','source','country','title','article']

In [192]:
df_11_cnn.head()

,file_name,date,source,country,title,article
0,cnn_11,2020-07-03,CNN,US,Hong Kong's security law could have a chilling...,London (CNN Business)Hong Kong insists its vib...


**Saving**<a id='2.6_save'></a>

In [193]:
cd

C:\Users\rands


Saving it to Excel.

In [194]:
# df = pd.DataFrame(reviewlist)

# index=False below so that we don't get the dataframe index on the side; we just use the excel index
df_11_cnn.to_csv('./_Capstone_Two_NLP/data/_news/cnn_11.csv', index=False)

print('Complete')

Complete
